# Habit Tracker App using meta-llama/Meta-Llama-3.1-70B-Instruct

This notebook is heavily based on this one: [https://github.com/curiousily/AI-Bootcamp/blob/master/16.llm-function-calling.ipynb](16.llm-function-calling.ipynb). Also do checkout the amazing [YouTube tutorial on tool use](https://www.youtube.com/watch?v=5XFqrAG0OIk) by its author. 

What is different in this version is basically the model and the backend using vLLM. The model by default comes with a `chat_template` that seems to have an error that is fixed in this code. Also the tool use API in vLLM doesn't seem to be 100% compatible with the `openai` client libraries yet. Because of these differences, this notebook used the completions API instead of the chat API and parses the answers manually



In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.44.0 openai==1.40.3 --progress-bar off

In [37]:
import json
from json.decoder import JSONDecodeError
import os
import sqlite3
from dataclasses import dataclass
from datetime import date
from enum import Enum, auto
from typing import List, Set

In [38]:
import json
import openai
openai.api_key = '...'
openai.base_url = "http://0.0.0.0:8000/v1/"
MODEL = "meta-llama/Meta-Llama-3.1-70B-Instruct"

In [ ]:
os.environ["HF_TOKEN"]=input()

In [39]:
DB_NAME = "habit_tracker.db"


class DayOfWeek(Enum):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()


@dataclass
class Habit:
    id: int
    name: str
    repeat_frequency: Set[DayOfWeek]
    tags: List[str]


@dataclass
class DailyHabitEntry:
    id: int
    name: str
    tags: List[str]
    is_completed: bool


def get_connection():
    return sqlite3.connect(DB_NAME)


def create_tables():
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS habits (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                repeat_frequency TEXT NOT NULL,
                tags TEXT NOT NULL
            )
        """
        )
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS completions (
                habit_id INTEGER,
                completion_date TEXT,
                PRIMARY KEY (habit_id, completion_date),
                FOREIGN KEY (habit_id) REFERENCES habits (id)
            )
        """
        )
        conn.commit()


def list_habits() -> List[Habit]:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM habits")
        return [
            Habit(
                id,
                name,
                {DayOfWeek[day] for day in freq.split(",")},
                tags.split(","),
            )
            for id, name, freq, tags in cursor.fetchall()
        ]


def habits_for_date(date: date) -> List[DailyHabitEntry]:
    weekday = DayOfWeek(date.weekday() + 1).name
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            SELECT h.id, h.name, h.tags, c.completion_date IS NOT NULL as completed
            FROM habits h
            LEFT JOIN completions c ON h.id = c.habit_id AND c.completion_date = ?
            WHERE instr(h.repeat_frequency, ?) > 0
        """,
            (date.isoformat(), weekday),
        )
        return [
            DailyHabitEntry(id, name, tags.split(","), bool(completed))
            for id, name, tags, completed in cursor.fetchall()
        ]


def complete_habit(habit_id: int, completion_date: date):
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT OR REPLACE INTO completions (habit_id, completion_date)
            VALUES (?, ?)
        """,
            (habit_id, completion_date.isoformat()),
        )
        conn.commit()


def add_habit(name: str, repeat_frequency: Set[DayOfWeek], tags: List[str] = []) -> int:
    with get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute(
            """
            INSERT INTO habits (name, repeat_frequency, tags)
            VALUES (?, ?, ?)
        """,
            (name, ",".join(day.name for day in repeat_frequency), ",".join(tags)),
        )
        conn.commit()
        return cursor.lastrowid


def show_habits_for_date(date: date):
    print(f"Habits for {date}:")
    for entry in habits_for_date(date):
        status = "Completed" if entry.is_completed else "Not completed"
        print(f"- {entry.name} (ID: {entry.id}): {status}")
        print(f"  Tags: {', '.join(entry.tags)}")


create_tables()

In [40]:
add_habit(
    "Hit the gym",
    {DayOfWeek.MONDAY, DayOfWeek.WEDNESDAY, DayOfWeek.FRIDAY},
    ["exercise", "fitness"],
)
add_habit("Feed the llamas", {DayOfWeek.SATURDAY, DayOfWeek.SUNDAY}, ["diet"])

2

In [41]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.MONDAY: 1>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SATURDAY: 6>, <DayOfWeek.SUNDAY: 7>}, tags=['diet'])]

In [42]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "add_habit",
            "description": "Add a new habit. Returns the id for the habit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the habit",
                    },
                    "repeat_frequency": {
                        "type": "array",
                        "description": "Days of week to repeat, e.g. ['MONDAY', 'WEDNESDAY', 'FRIDAY']",
                        "items": {
                            "type": "string",
                            "enum": [
                                "MONDAY",
                                "TUESDAY",
                                "WEDNESDAY",
                                "THURSDAY",
                                "FRIDAY",
                                "SATURDAY",
                                "SUNDAY",
                            ],
                        },
                    },
                    "tags": {
                        "type": "array",
                        "description": "List of tags, e.g. ['health', 'fitness']",
                    },
                },
                "required": ["name", "repeat_frequency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_habits",
            "description": "Returns a list of all available habits",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "habits_for_date",
            "description": "Returns a list of habits scheduled for a date",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "str",
                        "description": "Date for which to display scheduled habits in ISO format e.g. 2024-11-23",
                    }
                },
                "required": ["date"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "complete_habit",
            "description": "Completes a specific habit for a given date",
            "parameters": {
                "type": "object",
                "properties": {
                    "habit_id": {
                        "type": "integer",
                        "description": "Id of the habit, e.g. 1",
                    },
                    "completion_date": {
                        "type": "str",
                        "description": "Date for completion of the habit in ISO format e.g. 2024-11-23",
                    },
                },
                "required": ["habit_id", "completion_date"],
            },
        },
    },
]

In [43]:
AVAILABLE_FUNCTIONS = {
    "add_habit": add_habit,
    "list_habits": list_habits,
    "habits_for_date": habits_for_date,
    "complete_habit": complete_habit,
}

In [44]:
argument_mapping = {}
argument_mapping["repeat_frequency"] = lambda day_names: [
    DayOfWeek[d] for d in day_names
]
argument_mapping["date"] = lambda d: date.fromisoformat(d)
argument_mapping["completion_date"] = lambda d: date.fromisoformat(d)

In [45]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.chat_template = tokenizer.chat_template.replace('argument', 'parameter')


Here we're going to use the chat template documented at https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#json-based-tool-calling

In [46]:
user_question = "Add a new habit for Reading a book every weekday #learning"
messages = [
    {
        "role": "system",
        "content":  "You are a helpful assistant with tool calling capabilities."
                    " When you receive a tool call response, use the output to format"
                    " an answer to the orginal user question.use variables."
    },
    {
        "role": "user",
        "content": 'Given the following functions, please respond with a JSON for a function'
                   ' call with its proper arguments that best answers the given prompt.\n'
                   'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.'
                   ' Do not use variables.\n\n' +\
                   "\n\n".join([json.dumps(t, indent=4) for t in TOOLS]) +\
                   f'\nQuestion: {user_question}'
    },
    #{
    #    "role": "assistant",
    #    "tool_calls": [{"type": "function", "function": json.loads('{"name": "add_habit", "parameters": {"name": "Reading a book", "repeat_frequency": "[\\"MONDAY\\", \\"TUESDAY\\", \\"WEDNESDAY\\", \\"THURSDAY\\", \\"FRIDAY\\"]", "tags": "[\\"learning\\"]"}}')}],
    #},
    #{
    #    "role": "ipython",
    #    "content": {"output": 1},
    #},
]
instruction = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, tools_in_user_message=False)
print(instruction)
response = openai.completions.create(
    model=MODEL,
    prompt=instruction,
    temperature=0,
    max_tokens=3000,
)
response

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question.use variables.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{
    "type": "function",
    "function": {
        "name": "add_habit",
        "description": "Add a new habit. Returns the id for the habit.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the habit"
                },
              

Completion(id='cmpl-663ccac93d314b698babae18328b2fa9', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='{"name": "add_habit", "parameters": {"name": "Reading a book", "repeat_frequency": "[\\"MONDAY\\", \\"TUESDAY\\", \\"WEDNESDAY\\", \\"THURSDAY\\", \\"FRIDAY\\"]", "tags": "[\\"learning\\"]"}}', stop_reason=None, prompt_logprobs=None)], created=1725572073, model='meta-llama/Meta-Llama-3.1-70B-Instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=60, prompt_tokens=681, total_tokens=741))

In [47]:
tool_calls = [json.loads(choice.text) for choice in response.choices]
tool_calls

[{'name': 'add_habit',
  'parameters': {'name': 'Reading a book',
   'repeat_frequency': '["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY"]',
   'tags': '["learning"]'}}]

In [48]:
print(response.choices[0].text)

{"name": "add_habit", "parameters": {"name": "Reading a book", "repeat_frequency": "[\"MONDAY\", \"TUESDAY\", \"WEDNESDAY\", \"THURSDAY\", \"FRIDAY\"]", "tags": "[\"learning\"]"}}


In [49]:
for tool_call in tool_calls:
    function_name = tool_call["name"]
    function_to_call = AVAILABLE_FUNCTIONS[function_name]
    function_args = tool_call["parameters"] # Notice that the model likes to return "parameters" instead if "arguments"

In [50]:
# For some reason LLama 3.1 returns arrays serialized as strings
for k,v in function_args.items():
    try:
        if type(v) == str:
            function_args[k] = json.loads(v)
    except JSONDecodeError:
        pass

In [51]:
function_args

{'name': 'Reading a book',
 'repeat_frequency': ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY'],
 'tags': ['learning']}

In [52]:
arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
arguments_to_map

['repeat_frequency']

In [53]:
argument_mapping['repeat_frequency']

<function __main__.<lambda>(day_names)>

In [54]:
for arg_name in arguments_to_map:
    function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])

In [55]:
function_args

{'name': 'Reading a book',
 'repeat_frequency': [<DayOfWeek.MONDAY: 1>,
  <DayOfWeek.TUESDAY: 2>,
  <DayOfWeek.WEDNESDAY: 3>,
  <DayOfWeek.THURSDAY: 4>,
  <DayOfWeek.FRIDAY: 5>],
 'tags': ['learning']}

In [56]:
function_response = function_to_call(**function_args)
function_response

3

In [57]:
list_habits()

[Habit(id=1, name='Hit the gym', repeat_frequency={<DayOfWeek.MONDAY: 1>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.FRIDAY: 5>}, tags=['exercise', 'fitness']),
 Habit(id=2, name='Feed the llamas', repeat_frequency={<DayOfWeek.SATURDAY: 6>, <DayOfWeek.SUNDAY: 7>}, tags=['diet']),
 Habit(id=3, name='Reading a book', repeat_frequency={<DayOfWeek.FRIDAY: 5>, <DayOfWeek.MONDAY: 1>, <DayOfWeek.WEDNESDAY: 3>, <DayOfWeek.THURSDAY: 4>, <DayOfWeek.TUESDAY: 2>}, tags=['learning'])]

In [58]:
def map_arguments(function_args: dict, argument_mapping: dict = argument_mapping):
    arguments_to_map = list(set(function_args.keys()) & set(argument_mapping.keys()))
    for arg_name in arguments_to_map:
        function_args[arg_name] = argument_mapping[arg_name](function_args[arg_name])
    return function_args

In [59]:
from dataclasses import dataclass, asdict, is_dataclass

class DataClassEncoder(json.JSONEncoder):
    def default(self, obj):
        if is_dataclass(obj):
            return asdict(obj)
        return super().default(obj)

def call_model(messages):
    instruction = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, tools_in_user_message=False)
    #print(instruction)
    
    response = openai.completions.create(
        model=MODEL,
        prompt=instruction,
        temperature=0,
        max_tokens=3000,
    )

    assert len(response.choices) == 1, "Don't know how to handle multiple choices"
    return response.choices[0].text


def user_message(tools, user_query):
    # Note: here is an addition to the prompt because functions that don't return anything confuse the model.
    return {
        "role": "user",
        "content": 'Given the following functions, please respond with a JSON for a function'
                   ' call with its proper arguments that best answers the given prompt.\n'
                   'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.'
                   ' Do not use variables. If the function output is empty write an aswer based on the user query explaining what has been done.\n\n' +\
                   "\n\n".join([json.dumps(t, indent=4) for t in tools]) +\
                   f'\nQuestion: {user_query}'
    }

def call_function(prompt, messages: List) -> List:
    messages.append(user_message(TOOLS, prompt))
    
    response_message = call_model(messages)
    messages.append({
        "role": "assistant",
        "content": response_message
    })

    #print(f"{response_message=}")
    tool_calls = [json.loads(response_message)]

    for tool_call in tool_calls:
        function_name = tool_call["name"]
        function_to_call = AVAILABLE_FUNCTIONS[function_name]
        function_args = tool_call["parameters"]
        for k,v in function_args.items():
            try:
                if type(v) == str:
                    function_args[k] = json.loads(v)
            except JSONDecodeError:
                pass

        function_args = map_arguments(function_args, argument_mapping)

        function_response = function_to_call(**function_args)
        #print(f"{function_response=}")
        if function_response is not None:
            function_response = json.loads(json.dumps(function_response, cls=DataClassEncoder))
        else:
            function_response = {}
        messages.append(
            {
                "role": "tool",
                "content": {"output": function_response},
            }
        )
        response_message = call_model(messages)
        messages.append({"role": "assistant", "content": response_message})
    return messages

def start_messages():
    return [
        {
            "role": "system",
            "content":  "You are a helpful assistant with tool calling capabilities."
                        " When you receive a tool call response, use the output to format"
                        " an answer to the orginal user question.use variables."
        },
    ]


messages = start_messages()

In [60]:
today = date(2024, 7, 26)
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Not completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning


In [61]:
user_prompt = f"Show all habits for today - {today.isoformat()}"
result = call_function(user_prompt, messages)
print(result[-1]["content"])

Here are the habits scheduled for 2024-07-26:

1. Hit the gym (Tags: exercise, fitness) - Not completed
2. Reading a book (Tags: learning) - Not completed


In [62]:
user_prompt = f"Complete the gym habit for {today.isoformat()}"
result = call_function(user_prompt, start_messages())
print(result[-1]["content"])

The habit with id 1 (gym) has been completed for 2024-07-26.


In [63]:
show_habits_for_date(today)

Habits for 2024-07-26:
- Hit the gym (ID: 1): Completed
  Tags: exercise, fitness
- Reading a book (ID: 3): Not completed
  Tags: learning
